The Model Object - creating an object to manage all the network objects. This will help with reducing the repeatability of the code. I'm going to just walk through all the individual steps the book walks through instead of providing a summary of the whole model object.
- Overall having a model object makes it easier to store and load the model for future prediction and training; it will also make it easier to build new models by cutting down on coding required 

Step 1 - the basic setup:
- Create model object that is defined with an empty list attribute that holds the layers
- each layer is added to the list via an add method 
- add a set method that sets various features of the model such as the loss function and the type of optimizer to use
    - note: after the self parameter in the set method there is an "*" parameter. This makes it so that all of the following parameters are required to be defined by name explicilty. For example , you cannot just do set(BinaryCrossEntropy, Optimizer_SGD), it must be set(loss=BinaryCrossEntropy, optimizer = Optimizer_SGD). This to enforce better read-ability in the code

In [ ]:
class Model:
    def __init__(self):
        self.layers() ##holds all the layers that we then input using the add method
    def add(self, layer):
        self.layers.append(layer) #adds the layer to the self.layers list
    def set(self, *, loss, optimizer):
        self.loss = loss
        self.optimizer = optimizer

Step 2 - adding forward pass:
- there are a couple of intermediate steps that we have to do before we can do a full forward pass. First we must create a input layer that exists soley to be an object to facilitate the use of a for loop in the forward pass. The construction of this object, just makes it so that our inputs can now be referenced as a self.outputs to make the for loop work. Because all the other layers have a self.output attribute.
- then we must also create a finalize method that takes all the model layers we have added to the model object and sets them up for the forward and backward passes by assigning new facilitating attributes. The finalize method iterates over the layers list (which excludes the Layer_Input), and defines a new attribute for each layer class in our layers list. The attributes are either "prev" or "next", where prev is the previous layer and next is the next layer. For the first layer, the previous layer is the input layer; for the last layer, the next layer is the loss function. This is so that later in the code when we do forward pass, we can just iterate over the layers and call the layer.prev.output, which will get us the output of the previous layer. these prev and next references are references to the other layer objects to they don't create additional memory usage 

In [ ]:
#new input layer class

class Layer_Input:
    def forward(self, inputs):
        self.output = inputs #making so that we can reference the inputs as .output

In [ ]:
### Adding the finalize method

class Model:
    def __init__(self):
        self.layers() ##holds all the layers that we then input using the add method
    def add(self, layer):
        self.layers.append(layer) #adds the layer to the self.layers list
    def set(self, *, loss, optimizer):
        self.loss = loss
        self.optimizer = optimizer
    
    ###ADDING finalize method
    def finalize(self):
        self.input_layer = Layer_Input() #NOT INCLUDED IN THE layer_count below

        layer_count = len(self.layers)

        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]

            elif i < layer_count - 1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1] 
            
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss

Step 3: adding the forward method for forward pass
- this requires two additional methods - the forward method, and the train method. The train method is what actually iterates over the training data for the user defined epochs and it uses the model object's defined forward method.
- there is kind of a hierarchy of forward method calls here where the train method calls the model object's forward method which calls each layer object's forward method. The model's forward object uses layer.prev.output to pass the previous layer's outputs to the current layer, hence our need for the Layer Input class => our inputs need to have an .output attribute. The end of the model's forward method calls layer.output attribute for the network's final output. This is because the layer variable is defined in the for loop. So it's just taking the last layer from the for loop's output. Note that this does not include loss, which is defined later.
- the train function is largely just for show right now, it will have more stuff added later.

In [1]:
### Adding the finalize method

class Model:
    def __init__(self):
        self.layers() ##holds all the layers that we then input using the add method
    def add(self, layer):
        self.layers.append(layer) #adds the layer to the self.layers list
    def set(self, *, loss, optimizer):
        self.loss = loss
        self.optimizer = optimizer
    
    def finalize(self):
        self.input_layer = Layer_Input() #NOT INCLUDED IN THE layer_count below

        layer_count = len(self.layers)

        for i in range(layer_count):
            if i == 0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]

            elif i < layer_count - 1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1] 
            
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
    #ADDED this method
    def train(self, X, y, *, epochs = 1, print_every = 1):
        for epoch in range(1, epochs + 1):

            output = self.forward(X) # call the forward method of the model class

            print(output)
            exit()
    #ADDED this method
    def forward(self, X):

        self.input_layer.forward(X) #call the forward method of the actual layer class

        for layer in self.layers: #for each layer in our list of layers
            #this calls the forward function of the layer object
            layer.forward(layer.prev.output) #why we needed the new class Layer Input. So that we could call the output attribute of that layer 
        
        #last layer iterated in the for loop (aka the output layer)
        return layer.output #see the layer variable in the foor loop. Our final layer in the list is assigned as layer through the for loop, so can just call it
        #the loss is calculated in the training method above - not shown for this step currently.

Step 4: 